In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [291]:
pd.set_option("max_columns",25)

In [2]:
train1=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_1.csv')
train2=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_2.csv')
train3=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_3.csv')
train4=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_4.csv')
train5=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_5.csv')
train6=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_6.csv')
train7=pd.rewoad_csv('/Users/hemanth/Downloads/News/Train/Train_7.csv')
train8=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_8.csv')
train9=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_9.csv')
train10=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_10.csv')
train=pd.concat([train1,train2,train3,train4,train5,train6,train7,train8,train9,train10])

In [96]:
df1=pd.read_excel("/Users/hemanth/Downloads/News/Item_Category_Map/Item_Category_Map_1.xlsx")
df2=pd.read_excel("/Users/hemanth/Downloads/News/Item_Category_Map/Item_Category_Map_2.xlsx")
cat=pd.concat([df1,df2])

In [6]:
df1=pd.read_csv("/Users/hemanth/Downloads/News/Item_Attributes/Item_Attributes_1.csv")
df2=pd.read_csv("/Users/hemanth/Downloads/News/Item_Attributes/Item_Attributes_2.csv")
df3=pd.read_csv("/Users/hemanth/Downloads/News/Item_Attributes/Item_Attributes_3.csv")
att=pd.concat([df1,df2,df3])

In [24]:
test=pd.read_csv("/Users/hemanth/Downloads/News/sample-submission.csv")

In [3]:
word=pd.read_csv("/Users/hemanth/Downloads/News/word.csv")

/Users/hemanth/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
imp=pd.read_csv("/Users/hemanth/Downloads/News/imp_le.csv")

/Users/hemanth/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

ch=cat.copy()
gh=pd.concat([pd.Series(row['item_id'],row['Keywords'].split(',')) for _, row in ch.iterrows()]).reset_index()
gh1=pd.concat([pd.Series(row['item_id'],str(row['Concepts']).split(',')) for _, row in ch.iterrows()]).reset_index()

ch=pd.concat([gh,gh1])

ch.columns=["word","item_id"]
ch=pd.concat([pd.Series(row['item_id'],row['word'].split(' '))              
                    for _, row in ch.iterrows()]).reset_index()
ch.columns=["word","item_id"]

k=ch.iloc[3].word
li=(ch["word"]==k)
ch=ch[~li]

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

ch["word"]=ch['word'].apply(lambda x:  x if x not in stop else np.nan)
ch["word"]=ch['word'].apply(lambda x:  x if x!="The" else np.nan)
ch=ch.dropna()

ch["word"]= ch["word"].apply(lambda x: x.lower())

ch=ch.drop_duplicates()

li=(ch["word"]=="null")
ch=ch[~li]
word=ch.copy()

In [51]:
word=word.drop_duplicates()

In [5]:
Tag_distinct = word[["word","item_id"]].drop_duplicates()
DF =Tag_distinct.groupby(["word"], as_index = False, sort = False).count().rename(columns = {"item_id": "tag_count_DF"})[["word","tag_count_DF"]]
a=math.log10(len(np.unique(word["item_id"])))
DF["IDF"]=a-np.log10(DF["tag_count_DF"])
TF = pd.merge(word,DF,on = "word", how = "left", sort = False)

In [6]:
Vect_len=TF[["item_id","IDF"]]
Vect_len["TF-IDF-Sq"]=Vect_len["IDF"]**2
Vect_len =Vect_len.groupby(["item_id"], as_index = False, sort = False).sum().rename(columns = {"TF-IDF-Sq": "TF-IDF-Sq-sum"})[["item_id","TF-IDF-Sq-sum"]]
Vect_len["vect_len"] = np.sqrt(Vect_len[["TF-IDF-Sq-sum"]].sum(axis=1))
TF = pd.merge(TF,Vect_len,on = "item_id", how = "left", sort = False)
TF["TAG_WT"]=TF["IDF"]/TF["vect_len"]

TF=TF[["item_id","word","TAG_WT"]]
TF.columns=["item_id","word","wgt"]

/Users/hemanth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
data=pd.merge(user,TF,on=["item_id"],how="left")
data=data[["uuid","word","TAG_WT"]]
data=data.groupby(["uuid","word"],as_index=False).sum()

vect=data[["uuid","TAG_WT"]]
vect["tf_sq"]=vect["TAG_WT"]**2
vect=vect.groupby(["uuid"],as_index=False,sort=False).sum().rename(columns={"tf_sq":"tf_sq_sum"})[["uuid","tf_sq_sum"]]
vect["vect_len"] = np.sqrt(vect[["tf_sq_sum"]].sum(axis=1))
data=pd.merge(data,vect,on=["uuid"],how="left")
data["TAG_WT"]=data["TAG_WT"]/data["vect_len"]
data=data[["uuid","word","TAG_WT"]]

In [155]:
distinc_word=np.unique(data["item_id"])

In [206]:
Final=pd.DataFrame(columns=["item1","item2","pro"])
for item in distinc_word:
    item1_val=data[data["item_id"]==item]
    item1_val=item1_val[["item_id","word","TAG_WT"]]
    for item2 in distinc_word:
        if item2!=item:
            item2_val=data[data["item_id"]==item2]
            item2_val=item2_val.rename(columns={"item_id":"item_id2","TAG_WT":"TAG_WT2"})
            item2_val=item2_val[["item_id2","word","TAG_WT2"]]
            item_data=pd.merge(item1_val,item2_val,on="word",how="inner")
            item_data["product"]=item_data["TAG_WT"]*item_data["TAG_WT2"]
            su=0
            su=item_data["product"].sum()
            Final=Final.append({'item1':item, 'item2':item2,"pro":su}, ignore_index=True)
            

KeyboardInterrupt: 

In [3]:
df=pd.read_csv("/Users/hemanth/Downloads/News/df.csv")

/Users/hemanth/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_test=pd.read_csv("/Users/hemanth/Downloads/News/df_test.csv")

In [4]:
imp["uvh"]=imp["uvh"].str.replace("0-",",")
imp["uvh"]=imp["uvh"].str.replace("S",",")
imp["uvh"]=imp["uvh"].str.replace("-",",")

k=imp["uvh"].iloc[1]
li=(imp["uvh"]==k)
imp=imp[~li]

imp1=imp[["uuid","uvh"]]
imp1=imp1.drop_duplicates()

user=pd.DataFrame()
user=pd.concat([pd.Series(row['uuid'],row['uvh'].split(',')) for _, row in imp1.iterrows()]).reset_index()
user.columns=[["item_id","uuid"]]
user=user.drop_duplicates()
user["len"]=user["item_id"].str.len()
li=user["len"]==8
user1=user[li].copy()
user1.drop(["len"],axis=1,inplace=True)


ids=df["uuid"]
ids2=df_test["uuid"]
iids=pd.DataFrame()
iids["id"]=pd.concat([ids,ids2])
li=user1["uuid"].isin(iids["id"])
user1=user1[li]

user1.to_csv("/Users/hemanth/Downloads/News/user_item.csv",index=False)

In [ ]:
ust=df[["uuid","item_id"]]
ust2=df_test[["uuid","item_id"]]
ust=pd.concat([ust,ust2])
ust=ust.drop_duplicates()

li=ust["item_id"].isin(TF["item_id"])
ust=ust[li]

li=ust["uuid"].isin(data["uuid"])
ust=ust[li]

ust.item_id=ust.item_id.astype("str")
ust=pd.merge(ust,TF,on=["item_id"],how="left")
ust=pd.merge(ust,data,on=["uuid","word"],how="left")
ust=ust.dropna()

ust["dot"]=ust["wgt"]*ust["TAG_WT"]
ust.drop(["word","wgt","TAG_WT"],axis=1,inplace=True)

ust2=ust.groupby(["uuid","item_id"],as_index=False).sum()

In [271]:
ch=df_test[["impression_id","item_id"]]
ch=ch.drop_duplicates()
ch["click"]=np.zeros(len(ch))

check=test.merge(ch,on=["impression_id","item_id"],how="left")
li=check["click"].isnull()
check=check[li]

item=df[["item_id","per_item"]]
item=item.drop_duplicates()

check2=pd.merge(check,item,on=["item_id"],how='left')
li=check2["per_item"]
li[li>0.5]=2
li[li<0.5]=1
check2=check2.fillna(1)

check2.index=check.index

li=check2["per_item"]
li=li.astype("int")

l=sub["click"]
l.iloc[ind]=li

In [272]:
item=df[["item_id","per_item"]]
item=item.drop_duplicates()

sub2=pd.merge(sub,item,on=["item_id"],how="left")

chec=sub2[sub2["per_item"]>0.5]

li=chec.index
sub2["click"].iloc[li]=2

In [12]:
df.drop(["impression_id","refrenceUrl","time","hour","timestamp_impression","item_id","uuid","uvh","geo","site_id","adunit_id","no_uuid","per_uuid"],axis=1,inplace=True)

In [15]:
df_test.drop(["impression_id","refrenceUrl","time","hour","timestamp_impression","item_id","uuid","uvh","geo","site_id","adunit_id","no_uuid","per_uuid"],axis=1,inplace=True)

In [31]:
train=pd.read_csv("/Users/hemanth/Downloads/News/df.csv")

In [49]:
test=pd.read_csv("/Users/hemanth/Downloads/News/df_test.csv")

In [61]:
test2=pd.read_csv("/Users/hemanth/Downloads/News/final_df_test.csv")

In [ ]:
train=train[["uuid","item_id","timestamp_impression"]]
train["timestamp_impression"]=pd.to_datetime(train["timestamp_impression"])
train["timestamp_impression"]=train["timestamp_impression"].astype("str")
train['timestamp_impression']=train['timestamp_impression'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S.%f'))
train['Day']=train['timestamp_impression'].apply(lambda x:x.day)
train['hour']=train['timestamp_impression'].apply(lambda x:x.hour)
train['minute']=train['timestamp_impression'].apply(lambda x:x.minute)
train['second']=train['timestamp_impression'].apply(lambda x:x.second)

In [50]:
test=test[["uuid","item_id","timestamp_impression"]]
test["timestamp_impression"]=pd.to_datetime(test["timestamp_impression"])
test["timestamp_impression"]=test["timestamp_impression"].astype("str")
test['timestamp_impression']=test['timestamp_impression'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S.%f'))
test['Day']=test['timestamp_impression'].apply(lambda x:x.day)
test['hour']=test['timestamp_impression'].apply(lambda x:x.hour)
test['minute']=test['timestamp_impression'].apply(lambda x:x.minute)
test['second']=test['timestamp_impression'].apply(lambda x:x.second)

In [52]:
test["vis"]=0
train["vis"]=0

In [54]:
check=pd.concat([train,test])

In [67]:
check.shape

(18298218, 8)

In [59]:
check=check.sort_values(["Day","hour","minute","second"])

In [47]:
check=check.groupby(["uuid","item_id"],as_index=False).count().rename(columns = {"vis": "no_vis"})[["uuid","item_id","no_vis"]]